In [1]:
# загружаем модель с HaggingFace

In [4]:
# https://huggingface.co/ai-forever/sbert_large_mt_nlu_ru

In [66]:
! pip install optimum[exporters]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [60]:
from transformers import AutoTokenizer, AutoModel
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    emb =  sum_embeddings / sum_mask
    return emb / torch.linalg.norm(emb, dim=1).reshape(-1,1)


#Sentences we want sentence embeddings for
sentences = ['Привет! Как твои дела?',
             'А правда, что 42 твое любимое число?',
             'Здорово, рад видеть!']

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


In [62]:
sentence_embeddings

tensor([[ 0.0060, -0.0177, -0.0595,  ...,  0.0345, -0.0672, -0.0134],
        [ 0.0083, -0.0186, -0.0018,  ...,  0.0101, -0.0319,  0.0486],
        [ 0.0093, -0.0058, -0.0027,  ...,  0.0117, -0.0385, -0.0273]])

In [72]:
torch.norm(sentence_embeddings[0])

tensor(1.0000)

In [71]:
sum(sentence_embeddings[0] * sentence_embeddings[1])

tensor(0.2704)

In [67]:
! optimum-cli export onnx --model ai-forever/sbert_large_mt_nlu_ru model_onnx/

2023-12-21 13:04:48.832019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 13:04:48.832186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 13:04:48.995254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 13:04:52.210973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export to ONNX.
Automatic task detection to feature-extraction (possible synonyms are: default, mask-generation, sentence-similarity).
Using the export variant default. Available variants are:
    - defau

In [73]:
# Model NER

In [74]:
# https://huggingface.co/models?pipeline_tag=token-classification&language=ru&sort=trending

In [75]:
# https://huggingface.co/surdan/LaBSE_ner_nerel

In [76]:
# https://huggingface.co/surdan/LaBSE_ner_nerel/blob/main/Train_model.ipynb
# ноутбук с обучением модели

In [77]:
# https://huggingface.co/surdan/LaBSE_ner_nerel/blob/main/Inference.ipynb
# ноутбук с инференсом модели

In [78]:
# https://huggingface.co/datasets/surdan/nerel_short
# датасет для обучения